In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings(action='ignore')

data_dir = '/kaggle/input/assign/Week6_NN심화_Assignment/'

In [ ]:
Train = pd.read_csv(data_dir + 'Train.csv')
Test = pd.read_csv(data_dir + 'Test.csv')
submission = pd.read_csv(data_dir + 'submission.csv')

In [ ]:
X = Train.iloc[:, :-1].values
X = X.reshape(-1, 28, 28, 1)
y = Train.iloc[:, -1].values

X_test = Test.values
X_test = X_test.reshape(-1, 28, 28, 1)

In [ ]:
# 이미지 증강 이용 (색반전)

#이미지 증강에는 회전, 좌우변환, 색반전, 노이즈추가 ,명암 및 채도 조절 , 해상도조절 등이 있지만
#여기서는 색반전만 추가했습니다.
#색반전만 추가했음에도 92.8% 에서 94.1 %의 성능 향상을 확인할수 있었습니다.

In [ ]:
kite = pd.concat([Train.iloc[:, :-1]-255,Train.iloc[:,-1]],axis=1)

In [ ]:
pd.concat([Train,kite],axis=0,ignore_index=True).apply(abs)

In [ ]:
Train = pd.concat([Train,kite],axis=0,ignore_index=True).apply(abs)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 16, stratify = y, shuffle = True)

# 정규화
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [ ]:
BATCH_SIZE = 128
EPOCHS = 100

In [ ]:
initializer = keras.initializers.HeUniform(seed = 21)
model = keras.Sequential([

    keras.layers.Conv2D(32,kernel_size=(3,3),input_shape=(28,28,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(64,(3,3)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    

    keras.layers.MaxPooling2D(pool_size=2),
    
    keras.layers.Conv2D(32,(3,3)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    
    keras.layers.MaxPooling2D(pool_size=2),

    keras.layers.Flatten(),
    

    keras.layers.Dense(128, kernel_initializer=initializer),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.1),

    keras.layers.Dense(64, kernel_initializer=initializer),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),

    keras.layers.Dense(10, activation='softmax', kernel_initializer=initializer)
])




from keras.callbacks import ModelCheckpoint,EarlyStopping


early_stoping_callback = EarlyStopping(monitor='val_loss',patience=5,
                                      restore_best_weights=True)

print(model.summary())

In [ ]:
from keras.utils import multi_gpu_utils


optimizer = keras.optimizers.Adam(lr=0.01, decay = 0.001)
with tf.device('/gpu:0'):
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])


In [ ]:
result = model.fit(X_train, y_train, epochs=EPOCHS, 
                   validation_data = (X_val, y_val), batch_size=BATCH_SIZE,
                  callbacks=[early_stoping_callback])

In [ ]:
pred_label = np.argmax(model.predict(X_test), axis = 1)

submission['label'] = pred_label

submission.to_csv('/kaggle/working' + 'submission.csv', index = False)

submission.head()

In [ ]:
# k-fold 혹은 더 깊은 모델을 만들면 성능이 훨씬 증가할 것으로 예상됩니다.